# Домашнее задание 3: Natural Language Inference

In [1]:
# импортируем библиотеки
import pandas as pd

In [2]:
url = 'https://raw.githubusercontent.com/mcdm/CommitmentBank/master/CommitmentBank-items.csv'
df = pd.read_csv(url)

У некоторых глаголов, например, бывает только два или три оператора. Например, как у глагола accept только два оператора, и у глагола announce только три. Оставим в датасете только те глаголы, у которых есть операторы: modal, negation, conditional, question.

In [3]:
grouped = df.groupby('Verb')['Embedding.1'].nunique()
grouped

Verb
accept         2
admit          4
announce       3
assume         4
believe        4
bet            1
bother         1
convince       2
decide         2
demand         2
expect         2
fear           2
feel           4
figure         1
find           4
foresee        1
forget         3
guarantee      1
guess          3
hear           4
hope           3
hypothesize    1
imagine        4
insist         3
know           4
learn          3
mean           4
notice         4
occur          1
pretend        3
prove          2
realize        3
recognize      1
remember       3
say            4
see            4
seem           1
show           1
signal         2
specify        1
suggest        4
suppose        4
suspect        3
swear          1
take           1
tell           4
think          4
understand     3
Name: Embedding.1, dtype: int64

In [4]:
filtered_verbs = grouped[grouped == 4].index
filtered_verbs

Index(['admit', 'assume', 'believe', 'feel', 'find', 'hear', 'imagine', 'know',
       'mean', 'notice', 'say', 'see', 'suggest', 'suppose', 'tell', 'think'],
      dtype='object', name='Verb')

In [5]:
filtered_df = df[df['Verb'].isin(filtered_verbs)]
filtered_df

,Unnamed: 0,uID,Verb,Embedding,Context,Target,Prompt,ModalType,MatTense,MatSubjLemma,MatSubjPer,MatSubjNum,Embedding.1,genre,factive,mean.noTarget,sd.noTarget,Mean,SD,Reponses
0,1,BNC-1,admit,conditional,Polly had to think quickly.,They were still close enough to shore for him ...,Polly was not an experienced ocean sailor,NaN,future,she,third,singular,conditional,BNC,no,-0.111111,1.269296,2.000000,0.866025,"2, 3, 3, 1, 3, 1, 2, 1, 2"
1,2,BNC-1002,say,modal,"Nevertheless, life went on as it always does. ...",Indeed it could be said that they had prospered.,the Kiwi Keith and the Mackenzie houses had pr...,CI,present,it,third,singular,modal,BNC,no,1.200000,1.873796,1.875000,0.834523,"2, 1, 3, 1, 2, 2, 3, 1"
2,3,BNC-1003,say,modal,"He patted her hand. If he had chosen to, Thoma...",He might have said to her that some time in th...,some time in the middle of the nineteenth cent...,AB,present,he,third,singular,modal,BNC,no,-0.250000,0.462910,0.666667,1.154701,"2, 0, 1, 0, 0, 3, -1, 0, 1, 0, 0, 2"
3,4,BNC-1005,say,modal,She could see his distorted image in the slant...,Of course she could say it was for the childre...,it was for the children,AB,future,she,third,singular,modal,BNC,no,0.636364,1.120065,0.875000,0.991031,"2, 1, 0, 1, -1, 1, 1, 2"
4,5,BNC-1006,say,modal,"She glanced around the room, laying her hot fl...",Robyn swallowed and took a deep breath trying ...,it was all right,AB,future,she,third,singular,modal,BNC,no,-1.090909,1.044466,0.000000,2.309401,"3, 1, -2, -3, -3, 3, 2, 1, -1, -1"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1194,1195,WSJ-93,know,negation,The city's Campaign Finance Board has refused ...,"And, says Mr. Dinkins, he didn't know the man ...",the man Mr. Dinkins' campaign paid for a get-o...,NaN,past,he,third,singular,negation,WSJ,yes,-0.818182,1.401298,1.812500,1.470544,"3, 3, -2, 3, 3, 3, 3, 1, 2, 1, 1, 0, 3, 1, 3, 1"
1195,1196,WSJ-95,feel,negation,"Across the board, these consumers value qualit...",Twenty percent don't even feel they are financ...,they are financially well off,NaN,present,twenty percent,third,plural,negation,WSJ,no,-1.750000,1.035098,-1.000000,2.371708,"-2, 3, -3, -3, -2, -3, -3, 2, -3, -1, 3, -3, 0..."
1196,1197,WSJ-96,mean,negation,The Fed has assumed a similar responsibility i...,This does not mean that the Federal Reserve do...,the Federal Reserve does not already play an i...,NaN,present,this,third,singular,negation,WSJ,no,-0.722222,1.964555,0.125000,2.028957,"2, -2, 3, 0, 2, 1, 2, -2, 2, 1, -1, 0, -2, -3,..."
1198,1199,WSJ-98,believe,negation,"Talks between the two sides could unravel, of ...",And although Warner has said it wanted the pro...,the relationship could be repaired after the a...,NaN,present,they,third,plural,negation,WSJ,no,1.083333,1.378954,-1.733333,1.751190,"-1, -3, -3, -1, -2, 1, -3, -3, -3, -3, 2, -2, ..."


Посчитаем средний рейтинг комитмента по предикату (think, know, believe etc.) и по оператору (modal, negation, conditional, question) из отфильтрованного датасета.

In [6]:
average_commitment = filtered_df.groupby(['Verb', 'Embedding.1'])['Mean'].mean().reset_index()
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                       ):
    print(average_commitment)

       Verb  Embedding.1   Mean
0     admit  conditional  2.000
1     admit        modal  1.755
2     admit     negation -0.222
3     admit     question  0.556
4    assume  conditional  0.667
5    assume        modal  0.923
6    assume     negation  0.077
7    assume     question -0.896
8   believe  conditional -0.908
9   believe        modal  1.053
10  believe     negation -0.307
11  believe     question -1.145
12     feel  conditional  0.385
13     feel        modal  1.248
14     feel     negation -1.494
15     feel     question -0.060
16     find  conditional  0.796
17     find        modal  1.150
18     find     negation -1.889
19     find     question  0.804
20     hear  conditional  1.139
21     hear        modal  1.678
22     hear     negation -0.142
23     hear     question  0.500
24  imagine  conditional -1.319
25  imagine        modal -0.297
26  imagine     negation -0.070
27  imagine     question -1.556
28     know  conditional  1.318
29     know        modal  1.659
30     k

Затем построим график в plotly. На осих Ох должны быть отмечены глаголы, на оси Оу отмечены средние значения комитмента. А линии на графики должны относится к операторам

In [11]:
import plotly
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots

In [12]:
fig = px.line(average_commitment, x='Verb', y='Mean', color='Embedding.1', markers=True,
              labels={'Mean': 'Mean certainty', 'Verb': 'Verb', 'Embedding.1': 'Operators'},
              title='Mean certainty ratings by predicate and operator', width=950, height=600)

fig.show()

In [43]:
filtered_df.groupby(['Embedding.1'])['Mean'].mean()

Embedding.1
conditional    0.528545
modal          1.201118
negation      -0.799242
question       0.321266
Name: Mean, dtype: float64

Условный контекст: средний комитмент получился положительным, что говорит нам о том, что в условном контексте чаще созхраняются следствия. Вопросительный контекст положительный, меньше среднего комитмента у условного контекста. Значит, условный контекст более проективный, чем вопросительный. Контекст отрицание: средний комитмент отрицательный, говорит нам о том, что чаще всего нарушаются следствия в этом контексте. Модальный оператор набрал больше всех (1,2). Значит, модальный оператор(контекст) более проективен, чем все другие.

Задача 2


Стимул 1:
1. Источник: НКРЯ
2. Предикат: утверждать
3. Тип оператора: следствие
4. посылка: Произведение Э. Юнга было моментально переведено на немецкий, а на Локка ссылались французские юристы, отстаивая бессрочные права авторов на произведения. Так, в 1726 г. Луи д'Эрикур, ссылался на Локка, защищая в суде бессрочные привилегии для авторов книг.
5. целевое: "Он утверждал, что произведенное умом (сознанием) человека является плодом его труда, поэтому он имеет право свободно им распоряжаться в течение неограниченного времени (владеть собственными мыслями можно так же, как и землей, обработанной собственным трудом"
6. Рейтинг комитмента: +3

Комментарий: В данном случае, отношения свободятся к следствию.

Стимул 2:
1. Источник: НКРЯ
2. Предикат: признаться
3. Тип оператора:условный контекст
4. До целевого: И она это поняла, она, конечно же, это поняла! И не была такой дурой, чтобы ответить утвердительно. Врет Харченко: будь Георгий просто бродягой без дома, он бы его как раз и не взял.   
5. целевое предложение: если она признается, что он ей дорог, тут Георгию и каюк
6. Рейтинг комитмента: +1,5

Комментарий: Гипотеза, что он ей дорог, остается истинной, условный контекст не влияет на нее. Поэтому рейтинг комитмента положительный



Стимул 2:
1. Источник: НКРЯ
2. Предикат: подумать
3. Тип оператора: отрицание
4. Предложение до целевого: "Глядя на него..."
5. Целевое предложение: "никто бы не подумал, что этот невысокий, тщедушный на вид девятнадцатилетний паренек в недавнем прошлом студент Института физической культуры."
6. Рейтинг комитмента: +2

Комментарий: Использование двойного отрицания делает гипотезу, что он в прошлом студент Института физической культуры, истинной. Поэтому рейтинг положительный

Давайте создадим дополнительную колонку под названием Relation и будем записывать туда entailment, если значение комитмента находится в интервале [1,3]. Если значение комитмента находится в интервале (-1;1), то в колонку Relation записываем neutral. Если же значение комитмента находится в интервале [-3,-1], то в колонку Relation записываем contradiction.

In [16]:
def determine_relation(mean_value):
    if 1 <= mean_value <= 3:
        return 'entailment'
    elif -1 < mean_value < 1:
        return 'neutral'
    elif -3 <= mean_value <= -1:
        return 'contradiction'
    else:
        return 'unknown'

df['Relation'] = df['Mean'].apply(determine_relation)

После того как мы заполнили колонку Relation, давайте создадим колонку label. Если в колонке Relation строка равна entailment, то запишем 0 в колонку label. Если же равна neutral, то запишем 1 в колонку label. Если contradiction, то запишем 2


In [17]:
df['label'] = df['Relation'].map({'entailment': 0, 'neutral': 1, 'contradiction': 2})

Модель cointegrated/rubert-base-cased-nli-threeway обучалась разделять все три класса. В этой модели за основу взята нейросеть DeepPavlov/rubert-base-cased

In [13]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_checkpoint = 'cointegrated/rubert-base-cased-nli-threeway'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning:


The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.



tokenizer_config.json:   0%|          | 0.00/545 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.62M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/711M [00:00<?, ?B/s]

Мы добавляем маркер [CLS] для обозначения начала входных данных и маркер [SEP] для обозначения разделения между посылкой и гипотезой. Нам также нужно заполнить все входные данные, чтобы они были одинакового размера.

In [20]:
def predict_labels(target, prompt):
    input_sequence = f"[CLS] {target} [SEP] {prompt}"

    # Токенизация и конвертация в тензоры
    inputs = tokenizer(input_sequence, return_tensors="pt")

    # Предсказание
    with torch.no_grad():
        outputs = model(**inputs)

    # Извлечение логитов и предсказание класса
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=1).item()

    # Возвращение предсказанного класса (entailment, neutral, contradiction)
    return ['0', '1', '2'][predicted_class]

In [21]:
predict_labels(df['Target'][0], df['Prompt'][0])

'2'

In [26]:
df['Predicted_Labels'] = df.apply(lambda row: predict_labels(row['Target'], row['Prompt']), axis=1)

In [39]:
df['label'] = df['label'].apply(pd.to_numeric, errors="coerce")
df['Predicted_Labels'] = df['Predicted_Labels'].apply(pd.to_numeric, errors="coerce")
df.dtypes

Unnamed: 0            int64
uID                  object
Verb                 object
Embedding            object
Context              object
Target               object
Prompt               object
ModalType            object
MatTense             object
MatSubjLemma         object
MatSubjPer           object
MatSubjNum           object
Embedding.1          object
genre                object
factive              object
mean.noTarget       float64
sd.noTarget         float64
Mean                float64
SD                  float64
Reponses             object
Relation             object
label                 int64
Predicted_Labels      int64
dtype: object

In [40]:
true_labels = df['label']
predicted_labels = df['Predicted_Labels']

In [41]:
from sklearn.metrics import precision_score, recall_score, f1_score

precision = precision_score(true_labels, predicted_labels, average='weighted')
recall = recall_score(true_labels, predicted_labels, average='weighted')
f1 = f1_score(true_labels, predicted_labels, average='weighted')

# Вывод результатов
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

Precision: 0.2817780069643135
Recall: 0.32083333333333336
F1 Score: 0.2864498672210253


Если мы рассмотрим долю правильно предсказанных объектов среди всех предсказанных объектов, то мы получим метрику precision (точность). Интуитивно она показывает долю релевантных значений среди всех найденных значений. Чем меньше ложноположительных срабатываний будет допускать модель, тем больше будет её Precision. Наш результат показывает, что из предсказанных моделью случаев существенное количество ложных положительных результатов.

Метрика Recall: 0.32 - Модель обнаруживает только около 32% от общего числа реальных правильных случаев.

Метрика F1 Score: 0.29 - Среднегармоническое между precision и recall также довольно низкое, указывая на общую низкую производительность модели.

Можно сделать вывод, что BERT на данном датасете имеет ограниченную способность предсказания наличия следствия, противоречия или нейтрального отношения. Могу предположить, как можно улучшить наши результаты. Мы возьмем отдельный тренировочный тест, специально подготовленный для различения эти трех классов. Возьмем модель BERT. Я бы взяла saved model из библиотеки tensorflow hub. Взяла бы токенизатор и саму модель. После токенизировала все предложения в датасете, и записала их по такому образцу: [CLS] + [Promise] + [SEP] + [Hypothesis]. БЕРТ ожидает, что каждый пример будет начинаться с маркера "[CLS]", а предложения будут разделены маркером "[SEP]". Определим функцию построения модели, которая подключает слой классификатора softmax к выходу "[CLS]". Дальше начнем тренировать моделим, разделим на классы "0", "1", "2" в зависимости от оператора, и подадим на вход модели. И в конце предскажем для тестового набора данных. Тогда уже аккураси будет высокая, где-то между 80-90%.

Я пыталась это сделать в отдельной тетрадке (я тоже ее прикреплю), но при обучении модели у меня дальше не запускается (Это в гугл коллабе). Но когда работаю в VS code or Jupyter Notebook, то у меня не получается загрузить библиотеку tf-models-official (но это кажется связано с Pypy, наверное через надо было запускать). Я пыталась сделать по-другому: через base-bert-uncased, но там кажется не сходились по длине последовательности.  

Думаю, что я могу попробовать на других тренировачных и тестовых данных, и тогда все получится. Ведь входные данные будут одного размера, и можно будет предсказать уже для тестовой выборки. Тогда и метрики будут высокие.